In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Load dataset
data = pd.read_csv('/content/drive/MyDrive/dataset/data/recipes_only.csv')  # Ganti 'dataset.csv' dengan nama file dataset Anda
# Preprocessing data
# data['calories'] = data['calories'].apply(lambda x: x.replace(' kcal', '')).astype(float)
# data['fat'] = data['fat'].apply(lambda x: x.replace('g', '')).astype(float)
# data['sugar'] = data['sugar'].apply(lambda x: x.replace('g', '')).astype(float)

# # Calculate calories per unit
# data['calories_per_unit'] = data['calories'] / data['metric_serving_amount']

from sklearn.preprocessing import MinMaxScaler

# Normalize "metric_serving_amount" with consideration to calories per unit
# scaler = MinMaxScaler()
# data['metric_serving_amount_normalized'] = scaler.fit_transform(data[['metric_serving_amount']])

# # Adjust calories based on the normalized serving amount
# data['calorie_normalized'] = data['metric_serving_amount_normalized'] * data['calories_per_unit']

scaler = MinMaxScaler()
data['calorie_normalized'] = scaler.fit_transform(data[['calories']])

def determine_recipe_type(row):
    if row['recipe_types'] == 'Lunch':
        return 'Lunch'
    elif row['recipe_types'] == 'Breakfast':
        return 'Breakfast'
    else:
        return 'Dinner'

# Create 'meal_type' column
data['recipe_types'] = data.apply(determine_recipe_type, axis=1)

# Save updated dataset
data.to_csv('dataset_with_recipe_type.csv', index=False)

# Split data into features and labels
X = data[['calorie_normalized']]
y = data[['recipe_types', 'recipe_name','recipe_id']]  # Menambahkan kolom 'food_name' sebagai label

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize data
X_train_norm = (X_train - X_train.mean()) / X_train.std()
X_test_norm = (X_test - X_train.mean()) / X_train.std()

# Convert labels to one-hot encoding
y_train_encoded = pd.get_dummies(y_train['recipe_types'])
y_test_encoded = pd.get_dummies(y_test['recipe_types'])

# Define model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, activation='relu', input_shape=(1,)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train_norm, y_train_encoded, epochs=50, batch_size=32, verbose=1)

# Evaluate model
loss, accuracy = model.evaluate(X_test_norm, y_test_encoded, verbose=0)
print(f'Accuracy: {accuracy}')



Epoch 1/50
1709/1709 [==============================] - 4s 2ms/step - loss: 0.1846 - accuracy: 0.9651
Epoch 2/50
1709/1709 [==============================] - 3s 2ms/step - loss: 0.1678 - accuracy: 0.9662
Epoch 3/50
1709/1709 [==============================] - 3s 2ms/step - loss: 0.1677 - accuracy: 0.9662
Epoch 4/50
1709/1709 [==============================] - 3s 2ms/step - loss: 0.1669 - accuracy: 0.9662
Epoch 5/50
1709/1709 [==============================] - 3s 2ms/step - loss: 0.1667 - accuracy: 0.9662
Epoch 6/50
1709/1709 [==============================] - 3s 2ms/step - loss: 0.1662 - accuracy: 0.9662
Epoch 7/50
1709/1709 [==============================] - 3s 2ms/step - loss: 0.1652 - accuracy: 0.9662
Epoch 8/50
1709/1709 [==============================] - 3s 2ms/step - loss: 0.1652 - accuracy: 0.9662
Epoch 9/50
1709/1709 [==============================] - 3s 2ms/step - loss: 0.1647 - accuracy: 0.9662
Epoch 10/50
1709/1709 [==============================] - 3s 2ms/step - loss: 0.164

In [ ]:
# User input for maximum nutrient values per day
max_calorie = float(input("Enter maximum calorie per day: "))
# max_fat = float(input("Enter maximum fat per day: "))
# max_sugar = float(input("Enter maximum sugar per day: "))

# Normalize user input
user_input = pd.DataFrame([[max_calorie]], columns=['calorie_normalized'])
user_input_norm = (user_input - X_train.mean()) / X_train.std()

# Get meal recommendations
recommendations = model.predict(user_input_norm)
meal_types = y_train_encoded.columns.values

for i, recommendation in enumerate(recommendations[0]):
    print(f'Recommended {meal_types[i]}:')
    recommended_foods = y_train[(y_train['recipe_types'] == meal_types[i])][['recipe_id', 'recipe_name']]
    recommended_food_samples = recommended_foods.sample(n=5)
    for index, row in recommended_food_samples.iterrows():
        recipe_id = row['recipe_id']
        recipe_name = row['recipe_name']
        print(f'- Recipe ID: {recipe_id}, Name: {recipe_name}')

Enter maximum calorie per day: 2000
1/1 [==============================] - 0s 22ms/step
Recommended Breakfast:
- Food ID: 431411, Name: Breakfast Quesadillas
- Food ID: 7888137, Name: Bacon Breakfast Taquitos
- Food ID: 2608662, Name: Egg White Omelet with Spinach, Onion & Mushroom
- Food ID: 1183160, Name: Light French Toast
- Food ID: 6106024, Name: Broccoli BreakFAST
Recommended Dinner:
- Food ID: 27501, Name: Vegetable Coleslaw
- Food ID: 20767, Name: Strawberry Banana Shake
- Food ID: 766, Name: Juicy Chicken with Roasted Vegetables
- Food ID: 32344, Name: Spaghetti Squash with Turkey Meat Sauce
- Food ID: 50964, Name: Cinnamon Oatmeal Pancakes
Recommended Lunch:
- Food ID: 51023141, Name: Stir Fry Turkey with Rice
- Food ID: 75409, Name: Turkey Cranberry Wrap
- Food ID: 298, Name: Sourdough Chicken Club
- Food ID: 79142, Name: Spicy Beans and Rice
- Food ID: 30639, Name: Shwarma Chicken Wrap
